# Evaluates the performance of Logistic Regression Classifier with different training sizes of diferrent datasets

In [26]:
import pandas as pd


In [27]:
# GTZAN_dataset = pd.read_csv("../data/gtzan/features_30_sec.csv")
# GTZAN_dataset.drop(['filename','length'], axis =1, inplace = True)
# X = GTZAN_dataset.drop(['label'], axis=1)
# y = GTZAN_dataset['label']

In [28]:
fma = pd.read_csv('../data/fma/fmaManipulated.csv')
fma_small = fma[fma['subset'] == 'large']
fma_small.reset_index(inplace=True, drop=True)
fma_small.head()

,track_id,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6,subset,genre_top
0,135,0.978402,0.623572,1.129261,-1.043970,-1.127288,-0.032642,-0.878434,-0.916234,-0.743257,...,0.020464,-0.038451,0.234863,0.050837,0.050781,0.003418,0.297873,0.024899,large,Rock
1,137,1.607191,1.199649,0.745765,1.430835,-0.155455,0.768487,3.005594,2.779431,1.567088,...,0.011149,40.950294,0.619629,0.088439,0.086914,0.000000,3.275552,0.021831,large,Experimental
2,138,1.501403,0.957514,1.276685,0.640389,0.442936,0.845533,0.917440,0.770691,1.787260,...,0.012725,5.126394,0.627441,0.102232,0.099121,0.004883,0.633483,0.034676,large,Experimental
3,142,-0.581270,3.199484,1.298346,-0.681253,-0.935093,-0.960304,-0.748014,-0.715798,-0.905853,...,0.022975,4.865969,0.309570,0.066520,0.061035,0.001953,1.625022,0.036644,large,Folk
4,144,0.121207,-0.025308,0.541327,0.967687,1.082456,1.675841,1.741263,0.470098,4.051888,...,0.018568,0.627553,0.239258,0.079759,0.078613,0.000000,0.226211,0.034378,large,Jazz


In [29]:
X = fma_small.drop(['track_id','genre_top','subset'], axis=1)
y = fma_small['genre_top']

In [30]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import json

In [31]:
n_train = np.logspace(-2, np.log10(0.8), 10)
n_train

array([0.01      , 0.01627251, 0.02647945, 0.04308869, 0.0701161 ,
       0.11409647, 0.18566355, 0.30212113, 0.49162679, 0.8       ])

Arguments:

(X) Matrix of features

(y) Array of targets

(train_size) Size of the trainset, this trian set will be spllited into more partitions  

In [32]:
def performanceOfTrainSizes(X, y,):
    pipeline = Pipeline([
                            ('StandardScaler',  StandardScaler()),
                            ('meu_classificador', LogisticRegression(max_iter=10000))
                        ])

    n_train = np.logspace(-2, np.log10(0.8), 10)
    n_train = [int(n*X.shape[0]) for n in n_train]
    n_train = [n for n in n_train if n > len(set(y))]
    acc_mean = []
    acc_std = []
    acc_mean_overfit = []
    acc_std_overfit = []
    
    for n in n_train:
        this_acc = []
        this_acc_overfit = []
        for k in tqdm(range(50)):
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n, stratify=y)
            
            # accurracy
            pipeline.fit(X_train,y_train)
            y_pred = pipeline.predict(X_test)
            acc = accuracy_score(y_pred,y_test)
            # acurracy overfit

            y_pred_overfit = pipeline.predict(X_train)
            acc_overfit = accuracy_score(y_pred_overfit,y_train)

            this_acc_overfit.append(acc_overfit)
            this_acc.append(acc)
        
        this_acc_overfit = np.array(this_acc_overfit)
        this_acc = np.array(this_acc)
        acc_mean.append(this_acc.mean())
        acc_std.append(this_acc.std())
        acc_mean_overfit.append(this_acc_overfit.mean())
        acc_std_overfit.append(this_acc_overfit.std())
    return acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train

In [33]:
def saveMetricsStorage(storageJsonPath ,datasetName, acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train,datasetSize):
    with open(storageJsonPath, 'r') as f:
        storage = json.load(f)
    
    storage[datasetName] = {
        'acc_mean': acc_mean,
        'acc_mean_overfit': acc_mean_overfit,
        'acc_std': acc_std,
        'acc_std_overfit': acc_std_overfit,
        'n_train_': n_train,
        'datasetSize': datasetSize
    }
    
    with open(storageJsonPath, 'w') as f:
        json.dump(storage, f)

In [34]:

acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train = performanceOfTrainSizes(X, y)


  0%|          | 0/50 [00:00<?, ?it/s]

  6%|▌         | 3/50 [05:35<1:33:39, 119.57s/it]

In [ ]:
saveMetricsStorage('../data/newm.json', 'FMA-large', acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train,len(y))

In [ ]:
acc_mean

[0.6011931818181818,
 0.6136236346308921,
 0.6290154826283427,
 0.6355249076795538,
 0.6435101597642314,
 0.6460517746662324,
 0.6538452001416932,
 0.671544900279012,
 0.6916652482269502,
 0.7137346791636625]

In [ ]:
acc_mean_overfit 

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9993805309734514,
 0.9831779331779333,
 0.9348436380997849,
 0.8873239436619719,
 0.8440822214208439]

In [ ]:
acc_std

[0.012754033459509312,
 0.01206662781215089,
 0.009919569618706216,
 0.007993836772734682,
 0.008294186168222096,
 0.007763251155185104,
 0.005599720658258982,
 0.005363230813861691,
 0.004106284740186261,
 0.0071639674820494145]

In [ ]:
acc_std_overfit

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0008240752711874803,
 0.004448117379743556,
 0.005111608747337216,
 0.0037526578119127163,
 0.0024652530022986597]

In [ ]:
n_train

[138, 225, 367, 597, 972, 1582, 2574, 4189, 6816, 11092]